In [1]:
import selenium.webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement

driver = selenium.webdriver.Chrome()

In [2]:
site = "https://www.oldlistings.com.au/real-estate/VIC/Ascot+Vale/3032/buy/3/:bed:2:bedmax:2:bath:1:car:1:cat:Apartment"
driver.get(site)

In [3]:
table = driver.find_elements(By.CLASS_NAME, "content-col")[0]
table

<selenium.webdriver.remote.webelement.WebElement (session="63986d8b0ca7a2707c5f0db700b6a01e", element="f.A3811B32E6284619F545271B29AC7074.d.E7137077E2D18A9E87AF7DF2DC1ABBE9.e.39")>

In [4]:
listings = table.find_elements(By.TAG_NAME, "div")
listings

[<selenium.webdriver.remote.webelement.WebElement (session="63986d8b0ca7a2707c5f0db700b6a01e", element="f.A3811B32E6284619F545271B29AC7074.d.E7137077E2D18A9E87AF7DF2DC1ABBE9.e.40")>,
 <selenium.webdriver.remote.webelement.WebElement (session="63986d8b0ca7a2707c5f0db700b6a01e", element="f.A3811B32E6284619F545271B29AC7074.d.E7137077E2D18A9E87AF7DF2DC1ABBE9.e.37")>,
 <selenium.webdriver.remote.webelement.WebElement (session="63986d8b0ca7a2707c5f0db700b6a01e", element="f.A3811B32E6284619F545271B29AC7074.d.E7137077E2D18A9E87AF7DF2DC1ABBE9.e.41")>,
 <selenium.webdriver.remote.webelement.WebElement (session="63986d8b0ca7a2707c5f0db700b6a01e", element="f.A3811B32E6284619F545271B29AC7074.d.E7137077E2D18A9E87AF7DF2DC1ABBE9.e.42")>,
 <selenium.webdriver.remote.webelement.WebElement (session="63986d8b0ca7a2707c5f0db700b6a01e", element="f.A3811B32E6284619F545271B29AC7074.d.E7137077E2D18A9E87AF7DF2DC1ABBE9.e.43")>,
 <selenium.webdriver.remote.webelement.WebElement (session="63986d8b0ca7a2707c5f0db70

In [5]:
def extract_info(listing):
    address = ""
    try:
        information_list = listing.find_element(By.TAG_NAME, "section").find_elements(By.TAG_NAME, "section")

        general_info = extract_general(information_list[0])
        recent_price = extract_recent(information_list[1])
        historical_prices = extract_historical(information_list[2])

        extracted_data = {
            "general_info": general_info,
            "recent_price": recent_price,
            "historical_prices": historical_prices,
        }

        return extracted_data

    except:
        print(address, "FAILED")


def extract_recent(information: WebElement) -> dict[str, str | float | int]:
    """Extract recent prices."""
    date = information.find_element(By.TAG_NAME, "span").text.split(":")[-1]
    price = information.find_element(By.TAG_NAME, "h3").text.split(" ")[0]

    recent_price = {
        "date": date,
        "price": 0,
        "type": price,
    }

    return recent_price


def extract_general(information: WebElement) -> dict[str, str | float | int]:
    """Extract general information."""
    address = information.find_element(By.TAG_NAME, "h2").text
    bed = information.find_element(By.CLASS_NAME, "bed").text
    bath = information.find_element(By.CLASS_NAME, "bath").text
    car = information.find_element(By.CLASS_NAME, "car").text
    type = information.find_element(By.CLASS_NAME, "type").text

    general_info = {
        "address": address,
        "bed": bed,
        "bath": bath,
        "car": car,
        "type": type,
    }

    return general_info


def extract_historical(information: WebElement) -> list[dict[str, str | float | int]]:
    """Extract historical prices."""
    information_soup = BeautifulSoup(information.get_attribute("outerHTML"), "html.parser")
    historical_prices = information_soup.find_all("li")

    extracted_data = []
    for historical_price in historical_prices:
        date = historical_price.find("span").text
        market_info = historical_price.text.split(date)[1]
        extracted_data.append(
            {
                "date": date,
                "price": 0,
                "type": market_info,
            }
        )

    return extracted_data


listing = listings[0]
extract_info(listing)

{'general_info': {'address': '4/6 ORMOND ROAD, ASCOT VALE',
  'bed': '2',
  'bath': '1',
  'car': '1',
  'type': 'Unit/apmt'},
 'recent_price': {'date': ' January 2019', 'price': 0, 'type': '$495,000'},
 'historical_prices': [{'date': 'January 2019',
   'price': 0,
   'type': '$495,000'}]}

In [6]:
extracted_data = []
for listing in listings:
    extracted_data.append(extract_info(listing))
extracted_data

 FAILED
 FAILED
 FAILED
 FAILED
 FAILED
 FAILED
 FAILED
 FAILED
 FAILED
 FAILED
 FAILED
 FAILED
 FAILED
 FAILED
 FAILED
 FAILED
 FAILED
 FAILED
 FAILED
 FAILED
 FAILED


[{'general_info': {'address': '4/6 ORMOND ROAD, ASCOT VALE',
   'bed': '2',
   'bath': '1',
   'car': '1',
   'type': 'Unit/apmt'},
  'recent_price': {'date': ' January 2019', 'price': 0, 'type': '$495,000'},
  'historical_prices': [{'date': 'January 2019',
    'price': 0,
    'type': '$495,000'}]},
 {'general_info': {'address': '11/37 ASCOT VALE RD, ASCOT VALE',
   'bed': '2',
   'bath': '1',
   'car': '1',
   'type': 'Unit/apmt'},
  'recent_price': {'date': ' October 2018', 'price': 0, 'type': '$440'},
  'historical_prices': [{'date': 'October 2018',
    'price': 0,
    'type': '$440 Private Sale'}]},
 {'general_info': {'address': '7/1 LANGS ROAD, ASCOT VALE',
   'bed': '2',
   'bath': '1',
   'car': '1',
   'type': 'Unit/apmt'},
  'recent_price': {'date': ' September 2018', 'price': 0, 'type': '$430,000'},
  'historical_prices': [{'date': 'September 2018',
    'price': 0,
    'type': '$430,000 Private Sale'}]},
 {'general_info': {'address': '4/52 MUNRO STREET, ASCOT VALE',
   'bed':

In [7]:
historical_prices = [data["historical_prices"] for data in extracted_data if data is not None]
historical_prices

[[{'date': 'January 2019', 'price': 0, 'type': '$495,000'}],
 [{'date': 'October 2018', 'price': 0, 'type': '$440 Private Sale'}],
 [{'date': 'September 2018', 'price': 0, 'type': '$430,000 Private Sale'}],
 [{'date': 'July 2018', 'price': 0, 'type': 'By Negotiation'},
  {'date': 'October 2014', 'price': 0, 'type': '$320,000 - $350,000 Auction'}],
 [{'date': 'June 2018', 'price': 0, 'type': '$492,000'},
  {'date': 'June 2018', 'price': 0, 'type': 'Auction'},
  {'date': 'June 2018', 'price': 0, 'type': '$440,000 - $480,000'},
  {'date': 'February 2015', 'price': 0, 'type': '$455,000'},
  {'date': 'July 2012', 'price': 0, 'type': '$410,000 Price Guide'},
  {'date': 'June 2012', 'price': 0, 'type': '$415,000'},
  {'date': 'August 2012', 'price': 0, 'type': '$415,000'}],
 [{'date': 'May 2018', 'price': 0, 'type': '$350 Week'},
  {'date': 'February 2016', 'price': 0, 'type': '$360,000 - $395,000'}],
 [{'date': 'May 2018', 'price': 0, 'type': 'Private Sale'},
  {'date': 'May 2014', 'price': 

In [8]:
list_of_historical_prices = [historical_price["type"] for historical_price in sum(historical_prices, [])]
list_of_historical_prices

['$495,000',
 '$440 Private Sale',
 '$430,000 Private Sale',
 'By Negotiation',
 '$320,000 - $350,000 Auction',
 '$492,000',
 'Auction',
 '$440,000 - $480,000',
 '$455,000',
 '$410,000 Price Guide',
 '$415,000',
 '$415,000',
 '$350 Week',
 '$360,000 - $395,000',
 'Private Sale',
 '$400,000 - $440,000',
 '$400,000 - $440,000 Auction',
 'Auction',
 'Contact',
 'Auction',
 '$480,000 - $520,000',
 '$480,000 - $520,000 Auction',
 'Contact',
 '$625,000',
 '$649,950',
 '$649,950',
 'Contact',
 '$670,000',
 '$625,000',
 '$490,000 Private Sale',
 '$575,000',
 '$585,000',
 '$702,000',
 '$460,000 - $500,000 Auction',
 '$460,000 - $500,000',
 'Auction',
 'Contact',
 '$540,000 - $580,000',
 'Auction',
 '$480,000',
 '$465,000 Private Sale',
 '$440,000 - $480,000',
 '$300,000 In excess of',
 '$450,000 - $490,000 Auction',
 'Auction',
 '$450,000 - $490,000',
 '$410,000 - $450,000',
 'Private Sale',
 '$539,000 Private Sale',
 '$539,000 Private Sale',
 '$445,000 - $465,000 Private Sale',
 '$488,000 Priv

In [9]:
from enum import Enum


class RecordType(str, Enum):
    AUCTION = "auction"
    PRIVATE_SALE = "private_sale"
    ENQUIRY = "enquiry"
    NO_SALE = "no_sale"
    RENT = "rent"


def clean_market_info(market_info) -> tuple[float, RecordType]:
    pass

_IncompleteInputError: incomplete input (3369903208.py, line 12)